# **WEB SCRAPING TO GAIN COMPANY INSIGHTS**

# *TASK 1*


---
# Web scraping and analysis
Use a package called BeautifulSoup to colect data from the web. Once I've collected data and saved it into a local .csv file I should start with analysis.

# Scraping data from Skytax
Link to collect: [https://www.airlinequality.com/airline-reviews/british-airways] Use Python and BeautifulSoup to collect all the links to the reviews and then to collect the text data on each of individual review links.

In [1]:
# Import some libs

import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
# Create some lists to store data

rating10 = []
feedback = []
aircraft = []
travel_class = []
travel_type = []
route = []
date_flown = []
seat_comfort = []
cabin_staff_service = []
food_beverages = []
entertainment = []
wifi = []
ground_service = []
value_for_money = []
recommend = []
revId = []

In [3]:
# Function to collect data from web table

def value_fetcher(soup):
  aircraft.append(np.nan)
  travel_class.append(np.nan)
  travel_type.append(np.nan)
  route.append(np.nan)
  date_flown.append(np.nan)
  seat_comfort.append(np.nan)
  cabin_staff_service.append(np.nan)
  food_beverages.append(np.nan)
  entertainment.append(np.nan)
  wifi.append(np.nan)
  ground_service.append(np.nan)
  value_for_money.append(np.nan)
  recommend.append(np.nan)

  for row in soup.find_all('tr'):
    header = row.find('td', {'class': 'review-rating-header'}).text
    value = row.find('td', {'class': 'review-value'})
    if header == 'Aircraft':
      aircraft[-1] = value.text
    elif header == 'Type Of Traveller':
      travel_type[-1] = value.text
    elif header == 'Seat Type':
      travel_class[-1] = value.text
    elif header == 'Route':
      route[-1] = value.text
    elif header == 'Date Flown':
      date_flown[-1] = value.text
    elif header == 'Seat Comfort':
      seat_comfort[-1] = len(row.find_all('span', {'class': 'star fill'}))
    elif header == 'Cabin Staff Service':
      cabin_staff_service[-1] = len(row.find_all('span', {'class': 'star fill'}))
    elif header == 'Food & Beverages':
      food_beverages[-1] = len(row.find_all('span', {'class': 'star fill'}))
    elif header == 'Inflight Entertainment':
      entertainment[-1] = len(row.find_all('span', {'class': 'star fill'}))
    elif header == 'Ground Service':
      ground_service[-1] = len(row.find_all('span', {'class': 'star fill'}))
    elif header == 'Wifi & Connectivity':
      wifi[-1] = len(row.find_all('span', {'class': 'star fill'}))
    elif header == 'Value For Money':
      value_for_money[-1] = len(row.find_all('span', {'class': 'star fill'}))
    elif header == 'Recommended':
      recommend[-1] = value.text


In [4]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 30
page_size = 100

for i in range(1, pages + 1):
  url = f'{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}'
  response = requests.get(url)
  content = response.text
  soup = BeautifulSoup(content, 'html.parser')
  articles = soup.find_all('article', {'itemprop': 'review'})
  for article in articles:
    rating = article.find('div', {'itemprop': 'reviewRating'})
    rating = rating.find('span', {'itemprop': 'ratingValue'}).text if rating.find('span', {'itemprop': 'ratingValue'}) else 'NaN'
    rating10.append(rating)
    anchor = article.find('div', {'class': 'body'})
    id = anchor.attrs['id']
    id = id.replace('anchor', '', 1)
    revId.append(id)
    div = article.find('div', {'class': 'tc_mobile'})
    contents = div.find('div', {'class': 'text_content'}).text
    feedback.append(contents)
    table = div.find('table', {'class': 'review-ratings'})
    value_fetcher(table)

In [5]:
# Creating DataFrame from a dictionary of list
data = {
    'ReviewId': revId,
    'Rating10': rating10,
    'Review': feedback,
    'Aircraft': aircraft,
    'Travel_Type': travel_type,
    'Travel_Class': travel_class,
    'Route': route,
    'Date_flown': date_flown,
    'Seat_comfort': seat_comfort,
    'Cabin_staff_service': cabin_staff_service,
    'Food_beverages': food_beverages,
    'Inflight_entertainment': entertainment,
    'Wifi_connectivity': wifi,
    'Ground_service': ground_service,
    'Value_for_money': value_for_money,
    'Recommend': recommend
    }

In [6]:
df = pd.DataFrame(data)
df.head()

,ReviewId,Rating10,Review,Aircraft,Travel_Type,Travel_Class,Route,Date_flown,Seat_comfort,Cabin_staff_service,Food_beverages,Inflight_entertainment,Wifi_connectivity,Ground_service,Value_for_money,Recommend
0,862044,4,✅ Trip Verified | My family and I have flown ...,Boeing 777,Couple Leisure,Premium Economy,Chennai to London,July 2023,3.0,2.0,1.0,1.0,NaN,4.0,1,no
1,862019,2,✅ Trip Verified | This has been by far the wo...,A321neo,Couple Leisure,Economy Class,Istanbul to London,July 2023,2.0,2.0,NaN,NaN,NaN,1.0,1,no
2,861892,2,✅ Trip Verified | In Nov 2022 I booked and pa...,NaN,Solo Leisure,Economy Class,London to Edinburgh,November 2022,2.0,5.0,3.0,3.0,NaN,1.0,2,no
3,861589,4,Not Verified | BA is not treating its premium ...,Boeing 777 -200,Family Leisure,Premium Economy,Kingston to London,June 2023,5.0,4.0,4.0,3.0,NaN,3.0,3,no
4,861382,1,✅ Trip Verified | 24 hours before our departu...,Boeing 777-200,Couple Leisure,Economy Class,London Heathrow to Cape Town,June 2023,1.0,3.0,1.0,3.0,NaN,3.0,2,no


In [7]:
df.shape

(3000, 16)

In [8]:
# Saving DataFrame to CSV file
df.to_csv('British_Airway.csv', index = False)